<br>

<div align=center><font color=maroon size=6><b>0 Configuration</b></font></div>

<br>

In [1]:
import tensorflow as tf

<br>

In [5]:
tf.__version__

'2.5.0'

<br>

In [6]:
import os

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'                  # 指定第一块GPU可用

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5  # 程序最多只能占用指定 gpu 50%的显存
config.gpu_options.allow_growth = True                    #程序按需申请内存


sess = tf.compat.v1.Session(config = config)

# 参考链接：https://blog.csdn.net/weixin_44737922/article/details/118596152

<br>

参考文献：[Internal Error： Blas GEMM launch failed 问题](https://blog.csdn.net/feixiang7701/article/details/81515447)

```python
gpu_no = '0' # or '1'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
 
# 定义TensorFlow配置
config = tf.ConfigProto()
 
# 配置GPU内存分配方式，按需增长，很关键
config.gpu_options.allow_growth = True
 
# 配置可使用的显存比例
config.gpu_options.per_process_gpu_memory_fraction = 0.1
 
# 在创建session的时候把config作为参数传进去
sess = tf.InteractiveSession(config = config)

```

<br>

参考文献：[keras 或 tensorflow 调用GPU报错：Blas GEMM launch failed](https://blog.csdn.net/Leo_Xu06/article/details/82023330) `(注意：这篇文章使用的应该是 TF1 版)`

GPU版的tensorflow在模型训练时遇到Blas GEMM launch failed错误，或者keras遇到相同错误（keras 一般将tensorflow作为backend，如果安装了GPU版本的tensorflow，那么在使用keras时会优先使用GPU），类似报错如下：

<font size=2>`InternalError (see above for traceback): Blas GEMM launch failed : a.shape=(300, 1), b.shape=(1, 10), m=300, n=10, k=1`</font>

这是调用GPU时，显存分配遇到了问题，有些解释说当前电脑其他程序占用显存（游戏等），所以发生错误，具体讨论见[帖子](https://github.com/tensorflow/tensorflow/issues/11812)。

比较保险的方式是在模型训练之前为tensorflow或者keras分配显存空间:

① tensorflow就用如下语句创建session：

```python
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
```

② keras就在引入keras时进行参数设置：
```python
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()

#A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.allocator_type = 'BFC' 

config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True


set_session(tf.Session(config=config)) 

```

如果使用ipython notebook，做完上述设置后可能出现GPU sync failed，重启一下就应该没问题了。

<br>